<h1 align="center">An Introduction to Pandas</h1>
<h3 align="center">Shengwei Hou, Björn Grüning</h3>
<h4 align="center">Dec. 14th, 2015</h4>

<a id="Contents"></a>

# Contents:

1. [Series](#Series)
   * [Create a Series](#create_series)
   * [Series Operations](#series_operations)
2. [DataFrame](#DataFrame)
   * [Create a DataFrame](#create_dataframe)
   * [DataFrame Operations](#dataframe_operations)
       * [Data Overview](#data_overview)
       * [Sort the DataFrame](#sort_dataframe)
       * [Data Selection](#data_selection)
       * [Transform](#transform)
       * [Data Visualization](#visualization)
5. [Reference](#reference)

import modules

In [13]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

enable inline usage of matplotlib

In [14]:
%matplotlib inline

<a id='Series'></a>

# Series

[back to Contents](#Contents)

`Series` is a *one-dimensional labeled array* with axis labels that can host numbers, strings, Python objects etc. The axis labels are collectively referred to as the `index`. 

<a id='create_series'></a>

## Create a Series

[back to Contents](#Contents)

* `Series` can be created by passing a `list` or `numpy.array` instance. If no index is passed, a default list of integers: [0, ..., len(data) - 1] will be used as index. 

In [15]:
data = [1, 2, 3, 4, 5, np.nan, 'Tea']
index = ['row'+str(i) for i in range(1, 8)]

# pd.Series(data, index, dtype, name ...)
s = pd.Series(data=data, index=index)
s

row1      1
row2      2
row3      3
row4      4
row5      5
row6    NaN
row7    Tea
dtype: object

Note: If the data is a scalar value, it will be recycled/broadcasted to match the length of index. 

In [16]:
scalar = 5
index = ['row'+str(i) for i in range(1, 8)]
s = pd.Series(data=scalar, index=index)
s

row1    5
row2    5
row3    5
row4    5
row5    5
row6    5
row7    5
dtype: int64

* `Series` can also be created from `dict` instances, in this case, `keys` will be taken as `index`:

In [17]:
d = {'row1' : 0., 'row2' : 1., 'row3' : 2.}
s = pd.Series(d)
s

row1    0.0
row2    1.0
row3    2.0
dtype: float64

<a id='series_operations'></a>

## Series Operations

[back to Contents](#Contents)

* `numpy array` like operations

In [18]:
s[0:2] # slice a series

row1    0.0
row2    1.0
dtype: float64

In [19]:
s[['row1', 'row2']] # use index to select multiple elements

row1    0.0
row2    1.0
dtype: float64

In [20]:
s.std() # basic statistics also works like numpy array

1.0

In [21]:
s = s.append(pd.Series(data=99, index=['row4'])) # append a new item
s

row1     0.0
row2     1.0
row3     2.0
row4    99.0
dtype: float64

In [22]:
s.pop('row4') # pop one item by index

99.0

* `dict` like operations

In [23]:
s['row2']  # like query a dict

1.0

In [24]:
for k, v in s.iteritems(): # iterate over key-value pairs
    print k, v

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(k, v)? (<ipython-input-24-167fc430cdcd>, line 2)

In [25]:
s.keys() # keys are just indices

Index(['row1', 'row2', 'row3'], dtype='object')

In [26]:
s['row4'] = 100  # add a new item like dict
s

row1      0.0
row2      1.0
row3      2.0
row4    100.0
dtype: float64

In [27]:
del s['row4']  # del a item by index, raises KeyError if not found 
s

row1    0.0
row2    1.0
row3    2.0
dtype: float64

In [28]:
s.get('row10', 'default_value') # get value if index exists, otherwise return default value

'default_value'

<a id='DataFrame'></a>

#DataFrame

[back to Contents](#Contents)

A pandas `DataFrame` is a *2-dimensional labeled data structure* with columns of potentially different types.

<a id='create_dataframe'></a>

##Create a DataFrame

[back to Contents](#Contents)

* A  `pandas DataFrame` consists of a two-dimentional table (`data`) with row lables (`index`) and column lables (`columns`). 

In [29]:
index = ['row'+str(i) for i in range(1, 7)]
columns = ['column'+str(i) for i in range(1, 5)]
data = np.random.randn(6,4)

# pd.DataFrame(data, index, columns, dtype, ...)
df1 = pd.DataFrame(data=data, index=index, columns=columns)
df1

,column1,column2,column3,column4
row1,0.950154,1.363429,-0.436096,0.837200
row2,-1.315380,-0.071058,-0.701170,-0.206757
row3,-1.354902,-0.280704,0.064559,1.113632
row4,-1.448548,0.879852,0.794927,0.124480
row5,0.201061,-0.624937,0.910383,-0.356151
row6,-0.901838,1.048792,-0.400397,0.980808


* one can also create a `DataFrame` from a `dict` of `Series`

In [30]:
d = {'column1' : pd.Series([1., 2., 3.], index=['row1', 'row2', 'row3']),
     'column2' : pd.Series([1., 2., 3., 4.], index=['row1', 'row2', 'row3', 'row4'])}
df2 = pd.DataFrame(d)
df2

,column1,column2
row1,1.0,1.0
row2,2.0,2.0
row3,3.0,3.0
row4,NaN,4.0


* or from a `dict` of `list` or `numpy arrray`, in this case index needs to be specified

In [31]:
d = {'column1' : [1., 2., 3., 4.],
     'column2' : [4., 3., 2., 1.]}

df3 = pd.DataFrame(d, index=['row1', 'row2', 'row3', 'row4'])
df3

,column1,column2
row1,1.0,4.0
row2,2.0,3.0
row3,3.0,2.0
row4,4.0,1.0


<a id='dataframe_operations'></a>

##DataFrame Operations

[back to Contents](#Contents)

<a id='data_overview'></a>

### Data Overview

[back to Contents](#Contents)

In [32]:
index = ['row'+str(i) for i in range(1, 11)]
columns = ['column'+str(i) for i in range(1, 5)]
data = np.random.randn(10,4)

# pd.DataFrame(data, index, columns, dtype, ...)
df4 = pd.DataFrame(data=data, index=index, columns=columns)
df4

,column1,column2,column3,column4
row1,1.014568,0.259008,-1.618442,-0.289571
row2,-0.930709,-2.390890,-0.258501,-0.853898
row3,-1.053450,-0.025021,-0.481384,0.752868
row4,-0.562733,-0.793531,-0.665997,-0.087613
row5,0.193140,-0.124163,0.581899,-0.246235
row6,-0.078049,-0.358680,0.335342,-2.950201
row7,1.780316,0.840075,0.592133,0.760349
row8,-1.345681,0.765196,0.049689,-0.664016
row9,0.781289,0.154223,-0.971332,-0.595620
row10,0.346296,0.351105,0.215175,0.839512


* `head` to check the first `n` rows

In [33]:
# df.head(n)
df4.head()

,column1,column2,column3,column4
row1,1.014568,0.259008,-1.618442,-0.289571
row2,-0.930709,-2.390890,-0.258501,-0.853898
row3,-1.053450,-0.025021,-0.481384,0.752868
row4,-0.562733,-0.793531,-0.665997,-0.087613
row5,0.193140,-0.124163,0.581899,-0.246235


* `tail` to check the last `n` rows

In [34]:
# df.tail(n)
df4.tail(4)

,column1,column2,column3,column4
row7,1.780316,0.840075,0.592133,0.760349
row8,-1.345681,0.765196,0.049689,-0.664016
row9,0.781289,0.154223,-0.971332,-0.595620
row10,0.346296,0.351105,0.215175,0.839512


* `index` shows the row labels

In [35]:
df4.index     

Index(['row1', 'row2', 'row3', 'row4', 'row5', 'row6', 'row7', 'row8', 'row9',
       'row10'],
      dtype='object')

* `columns` shows the column lables

In [36]:
df4.columns   

Index(['column1', 'column2', 'column3', 'column4'], dtype='object')

* `values` shows the data contents

In [37]:
df4.values    

array([[ 1.01456792,  0.25900817, -1.61844205, -0.28957066],
       [-0.93070895, -2.39089043, -0.25850129, -0.85389765],
       [-1.05344963, -0.02502134, -0.48138362,  0.75286768],
       [-0.5627327 , -0.79353097, -0.66599678, -0.08761317],
       [ 0.19313955, -0.12416278,  0.58189891, -0.24623454],
       [-0.07804873, -0.35868049,  0.33534227, -2.95020137],
       [ 1.78031632,  0.84007457,  0.59213302,  0.76034874],
       [-1.34568133,  0.76519571,  0.04968866, -0.66401586],
       [ 0.7812887 ,  0.15422257, -0.97133198, -0.59562009],
       [ 0.34629557,  0.35110469,  0.21517508,  0.83951164]])

* `describe` shows a quick statistic summary

In [38]:
df4.describe()

,column1,column2,column3,column4
count,10.000000,10.000000,10.000000,10.000000
mean,0.014499,-0.132268,-0.222142,-0.333443
std,1.004103,0.932213,0.719717,1.111483
min,-1.345681,-2.390890,-1.618442,-2.950201
25%,-0.838715,-0.300051,-0.619843,-0.646917
50%,0.057545,0.064601,-0.104406,-0.267903
75%,0.672540,0.328081,0.305300,0.542747
max,1.780316,0.840075,0.592133,0.839512


* `df.mean` calculates mean on axis, also works for other `numpy`-derived methods

In [39]:
# df.mean(axis, skipna, numeric_only)
df4.mean(axis=0)

column1    0.014499
column2   -0.132268
column3   -0.222142
column4   -0.333443
dtype: float64

* `T` returns a transposed `DataFrame`

In [40]:
df4.T

,row1,row2,row3,row4,row5,row6,row7,row8,row9,row10
column1,1.014568,-0.930709,-1.053450,-0.562733,0.193140,-0.078049,1.780316,-1.345681,0.781289,0.346296
column2,0.259008,-2.390890,-0.025021,-0.793531,-0.124163,-0.358680,0.840075,0.765196,0.154223,0.351105
column3,-1.618442,-0.258501,-0.481384,-0.665997,0.581899,0.335342,0.592133,0.049689,-0.971332,0.215175
column4,-0.289571,-0.853898,0.752868,-0.087613,-0.246235,-2.950201,0.760349,-0.664016,-0.595620,0.839512


* `apply` functions to DataFrame 

In [41]:
# df.apply(func, axis, broadcast ...)
df4.apply(func=np.mean, axis=1)

row1    -0.158609
row2    -1.108500
row3    -0.201747
row4    -0.527468
row5     0.101160
row6    -0.762897
row7     0.993218
row8    -0.298703
row9    -0.157860
row10    0.438022
dtype: float64

<a id='sort_dataframe'></a>

### Sort the DataFrame

[back to Contents](#Contents)

* sort by an axis

In [42]:
# df.sort(columns, axis, ascending, inplace, kind, ...)
df4.sort(axis=0, ascending=True) # sort by rows, equivalent to df4.sort_index()

AttributeError: 'DataFrame' object has no attribute 'sort'

In [43]:
df4.sort(axis=1, ascending=False)  # sort by columns

AttributeError: 'DataFrame' object has no attribute 'sort'

* Sorting by values

In [44]:
df4.sort(columns='column4', ascending=True)  # sort by 'column4', ascendingly

AttributeError: 'DataFrame' object has no attribute 'sort'

In [45]:
df4.column4 = [2,2,2,2,2,1,1,1,1,1]
df4.sort(columns=['column4','column1'], ascending=[False, True])  # nested sort

AttributeError: 'DataFrame' object has no attribute 'sort'

<a id='data_selection'></a>

### Data Selection

[back to Contents](#Contents)

* select a column, yields a `Series` instance

In [ ]:
df4['column1'] # or df4.column1

* select multiple columns, yield a `DataFrame` instance

In [ ]:
df4[['column1', 'column2'] ]

* use `[]` to slice rows

In [ ]:
df4[0:3]

* use `df.loc` to get data by `index` or `columns` labels

In [ ]:
df4.loc['row1':'row5', 'column1':'column4' ]

In [ ]:
df4.loc['row1':'row5', ['column1','column4'] ]

* use `df.at` to get a scalar value

In [ ]:
df4.at['row1', 'column1']    # or df4.loc['row1', 'column1']

* use `df.iloc` select `index` and `columns` integer positions

In [ ]:
df4.iloc[1:4, 1:4]

* use df.iat select a scalar value by `index` and `columns` integer positions

In [ ]:
df4.iat[1,1]

* Boolean/Logical Indexing

In [ ]:
df4[df4.column4 ==1] # where the value of column4 equal to 1

In [ ]:
df4['column5'] = ['T1', 'T2', 'T3', 'T1', 'T2', 'T3', 'T1', 'T2', 'T3', 'T4']
df4

In [ ]:
df4[df4['column5'].isin(['T1', 'T4'])] # use isin()

In [ ]:
selected = df4.sum() > 3  # select columns whose sum more than 3
selected

In [ ]:
df4.loc[:,selected]       # select the filtered columns

<a id='transform'></a>

### Transform

[back to Contents](#Contents)

* `pd.concat` concatenates `DataFrame` objects 

In [ ]:
df1 = pd.DataFrame(np.random.randn(2, 4), index=['df1_row1', 'df1_row2'])
df2 = pd.DataFrame(np.random.randn(2, 5), index=['df2_row1', 'df2_row2'])
df3 = pd.DataFrame(np.random.randn(2, 4), index=['df3_row1', 'df3_row2'])

# pd.concat(objs, axis, ...)
df = pd.concat([df1, df2, df3])
df

* `pd.merge` will join two `DataFrame` objects based on common columns  

In [ ]:
left = pd.DataFrame({'key': ['row1', 'row2', 'row3'], 'lval': [1, 2, 3]})
right = pd.DataFrame({'key': ['row1', 'row2'], 'rval': [4, 5]})

# pd.merge(left, right, how, on, ...)
pd.merge(left, right, on='key', how='left')

* `df.append` returns a new `DataFrame` with new rows appended

In [ ]:
# df.append(other, ignore_index, ...)
df.append(df.iloc[0:3,])

<a id='visualization'></a>

### Data Visualization

[back to Contents](#Contents)

The `plot` method on `Series` and `DataFrame` is just a wrapper around `plt.plot`:

In [ ]:
sigma = 5
mu = 40
gc_content = sigma*np.random.randn(100) + mu    # create a gc content array, with mean=40, std=sigma

# plot gc content
s = pd.Series(gc_content, index=range(0, 100))
s.plot()

# add a red line to the mean
plt.axhline(y=40, linewidth=4, color='r')

In [ ]:
df = pd.DataFrame(np.random.randn(100, 4), index=s.index, columns=['sp1', 'sp2', 'sp3', 'sp4'])
df = df.cumsum() # plot the cumsum, to make each curve more distinct
df.plot()

In [ ]:
df = pd.DataFrame(np.random.randn(10, 4), columns=['sp1', 'sp2', 'sp3', 'sp4'])

# kind :'line', 'bar', 'barh', 'kde', 'density', 'area', 'scatter', 'hexbin'
df.plot(kind='bar')

In [ ]:
df.plot(kind='bar', stacked=True)

In [ ]:
df.plot(kind='density')

In [ ]:
df = pd.DataFrame(np.random.rand(10, 5), columns=['A', 'B', 'C', 'D', 'E'])
df.boxplot()

<a id='reference'></a>

# Reference

[back to Contents](#Contents)

1. 10 minutes to `pandas`: http://pandas.pydata.org/pandas-docs/stable/10min.html
2. `pandas` documentation: http://pandas.pydata.org/pandas-docs/stable/dsintro.html